<a href="https://colab.research.google.com/github/cmunozcortes/home-credit-default-project/blob/main/Project_AllDataSources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Data Cleaning
Here we will include data from all the sources available.
Let's start with `bureau.csv` and `bureau_balance.csv`. 
*  `bureau.csv`: information about client's previous loans with other financial institutions reported to Home Credit. Each previous loan has its own row.
*  `bureau_balance.csv`: monthly information about the previous loans. Each month has its own row.

In [1]:
import pandas as pd
import os
import gc
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

# List files in the data dir
os.listdir('drive/MyDrive/CS 249 Project/Data/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'previous_application.csv',
 'sample_submission.csv',
 'HomeCredit_columns_description.gsheet']

Some helper functions to optimize memory usage

In [2]:
def convert_types(df, print_info=False):
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [3]:
def del_feat_miss_val(df, threshold=0.5):
  df_miss = pd.DataFrame(df.isnull().sum())
  df_miss['percent'] = df_miss[0] / len(df)
  feat_miss_vals = list(df_miss.index[df_miss['percent'] > threshold])
  df = df.drop(columns=feat_miss_vals)
  return df

In [4]:
bureau = pd.read_csv('drive/MyDrive/CS 249 Project/Data/bureau.csv')
bureau = convert_types(bureau, print_info=True)
bureau_bal = pd.read_csv('drive/MyDrive/CS 249 Project/Data/bureau_balance.csv')
bureau_bal = convert_types(bureau_bal, print_info=True)

train = pd.read_csv('drive/MyDrive/CS 249 Project/Data/application_train.csv')
train = convert_types(train, print_info=True)

Original Memory Usage: 0.23 gb.
New Memory Usage: 0.1 gb.
Original Memory Usage: 0.66 gb.
New Memory Usage: 0.25 gb.
Original Memory Usage: 0.3 gb.
New Memory Usage: 0.13 gb.


## Functions to aggregate the data

### Numerical features

In [5]:
def agg_numeric(df, groupby_var, df_name):
  # Remove ID columns other than the grouping feature
  for col in df:
    if col != groupby_var and 'SK_ID' in col:
      df = df.drop(columns=col)
  
  # Get numeric features only
  numeric_df = df.select_dtypes('number').copy()

  # Add ID column to the numeric feature df
  numeric_df[groupby_var] = df[groupby_var]

  # Group by the specified variable and aggregate the statistical data
  agg = numeric_df.groupby(groupby_var)\
                  .agg(['count', 'mean', 'max', 'min', 'sum'])
  
  # Flatten multi-level columns
  agg.columns = agg.columns.to_flat_index()\
                   .map(lambda x: df_name+'_'+'_'.join(x).rstrip('_')\
                        if x[0] != groupby_var else x[0])
  
  # Remove columns with redundant values
  _, idx = np.unique(agg, axis=1, return_index=True)
  agg = agg.iloc[:, idx]

  gc.enable()
  del df, numeric_df
  gc.collect()

  return agg

### Categorical features
Aggregate categorical features
1. One-hot encode categorical features
2. Aggregate using mean and sum
3. Flatten multi-level column names

In [6]:
def agg_categorical(df, groupby_var, df_name):
  categorical = pd.get_dummies(df.select_dtypes('category')).copy()
  categorical[groupby_var] = df[groupby_var]
  categorical = categorical.groupby(groupby_var).agg(['sum', 'mean'])
  col_names = []
  for feature in categorical.columns.levels[0]:
    for stat in ['count', 'count_norm']:
      col_names.append(f'{df_name}_{feature}_{stat}')
  categorical.columns = col_names

  # Remove duplicate columns by values
  _, idx = np.unique(categorical, axis = 1, return_index = True)
  categorical = categorical.iloc[:, idx]

  gc.enable()
  del df
  gc.collect()

  return categorical

## Aggregate the data

### `bureau.csv`

In [7]:
# Aggregate the `bureau.csv` data
bureau_agg_cat = agg_categorical(bureau, 'SK_ID_CURR', 'bureau')
bureau_agg_num = agg_numeric(bureau.drop(columns='SK_ID_BUREAU'), 'SK_ID_CURR',
                             'bureau')

### `bureau_balance.csv`

In [8]:

# Aggregate the `bureau_balance.csv` data
bureau_bal_agg_cat = agg_categorical(bureau_bal, 'SK_ID_BUREAU', 'bureau_balance')
bureau_bal_agg_num = agg_numeric(bureau_bal, 'SK_ID_BUREAU', 'bureau_balance')

In [9]:
bureau_bal_agg_num.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count
SK_ID_BUREAU,,,,,
5001709,-4656,-96,-48.0,0,97
5001710,-3403,-82,-41.0,0,83
5001711,-6,-3,-1.5,0,4
5001712,-171,-18,-9.0,0,19
5001713,-231,-21,-10.5,0,22


In [10]:
# Merge bureau_balance.csv data (numerical + categorical data)
bureau_by_loan = bureau_bal_agg_num.merge(bureau_bal_agg_cat, right_index=True, 
                                          left_on='SK_ID_BUREAU', how='outer')
bureau_by_loan.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_2_count_norm,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_3_count,bureau_balance_STATUS_2_count,bureau_balance_STATUS_5_count,bureau_balance_STATUS_1_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_0_count,bureau_balance_STATUS_X_count_norm,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_C_count
SK_ID_BUREAU,,,,,,,,,,,,,,,,,,,,,
5001709,-4656,-96,-48.0,0,97,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,0.113402,0.886598,11,86
5001710,-3403,-82,-41.0,0,83,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.060241,5,0.361446,0.578313,30,48
5001711,-6,-3,-1.5,0,4,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.750000,3,0.250000,0.000000,1,0
5001712,-171,-18,-9.0,0,19,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.526316,10,0.000000,0.473684,0,9
5001713,-231,-21,-10.5,0,22,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,1.000000,0.000000,22,0


In [11]:
# Merge to include the SK_ID_CURR
client_loan_info = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']]
bureau_by_loan = bureau_by_loan.merge(client_loan_info, on='SK_ID_BUREAU', 
                                        how='left')
bureau_by_loan.head()

,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_2_count_norm,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_3_count,bureau_balance_STATUS_2_count,bureau_balance_STATUS_5_count,bureau_balance_STATUS_1_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_0_count,bureau_balance_STATUS_X_count_norm,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_C_count,SK_ID_CURR
0,5001709,-4656,-96,-48.0,0,97,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,0.113402,0.886598,11,86,NaN
1,5001710,-3403,-82,-41.0,0,83,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.060241,5,0.361446,0.578313,30,48,162368.0
2,5001711,-6,-3,-1.5,0,4,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.750000,3,0.250000,0.000000,1,0,162368.0
3,5001712,-171,-18,-9.0,0,19,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.526316,10,0.000000,0.473684,0,9,162368.0
4,5001713,-231,-21,-10.5,0,22,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,1.000000,0.000000,22,0,150635.0


In [12]:
# Aggregate the stats for each client
bureau_balance_by_client = agg_numeric(
    bureau_by_loan.drop(columns=['SK_ID_BUREAU']), 
    groupby_var='SK_ID_CURR', 
    df_name='client')

bureau_balance_by_client.head()

,client_bureau_balance_MONTHS_BALANCE_sum_sum,client_bureau_balance_MONTHS_BALANCE_sum_min,client_bureau_balance_MONTHS_BALANCE_sum_mean,client_bureau_balance_MONTHS_BALANCE_min_sum,client_bureau_balance_MONTHS_BALANCE_mean_sum,client_bureau_balance_MONTHS_BALANCE_min_min,client_bureau_balance_MONTHS_BALANCE_mean_min,client_bureau_balance_MONTHS_BALANCE_min_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_sum_max,client_bureau_balance_MONTHS_BALANCE_min_max,client_bureau_balance_MONTHS_BALANCE_mean_max,client_bureau_balance_MONTHS_BALANCE_max_sum,client_bureau_balance_MONTHS_BALANCE_max_min,client_bureau_balance_MONTHS_BALANCE_max_mean,client_bureau_balance_MONTHS_BALANCE_max_max,client_bureau_balance_STATUS_5_count_norm_min,client_bureau_balance_STATUS_5_count_min,client_bureau_balance_STATUS_4_count_norm_min,client_bureau_balance_STATUS_4_count_min,client_bureau_balance_STATUS_3_count_norm_min,client_bureau_balance_STATUS_3_count_min,client_bureau_balance_STATUS_4_count_norm_mean,client_bureau_balance_STATUS_5_count_norm_mean,client_bureau_balance_STATUS_4_count_norm_max,client_bureau_balance_STATUS_4_count_norm_sum,client_bureau_balance_STATUS_5_count_norm_max,client_bureau_balance_STATUS_5_count_norm_sum,client_bureau_balance_STATUS_4_count_mean,client_bureau_balance_STATUS_5_count_mean,client_bureau_balance_STATUS_4_count_max,client_bureau_balance_STATUS_4_count_sum,client_bureau_balance_STATUS_5_count_max,client_bureau_balance_STATUS_5_count_sum,client_bureau_balance_STATUS_2_count_norm_min,client_bureau_balance_STATUS_2_count_min,client_bureau_balance_STATUS_3_count_norm_mean,client_bureau_balance_STATUS_3_count_norm_max,client_bureau_balance_STATUS_3_count_norm_sum,client_bureau_balance_STATUS_3_count_mean,...,client_bureau_balance_STATUS_2_count_norm_max,client_bureau_balance_STATUS_2_count_norm_sum,client_bureau_balance_STATUS_2_count_mean,client_bureau_balance_STATUS_2_count_max,client_bureau_balance_STATUS_2_count_sum,client_bureau_balance_STATUS_X_count_norm_min,client_bureau_balance_STATUS_X_count_min,client_bureau_balance_STATUS_C_count_norm_min,client_bureau_balance_STATUS_C_count_min,client_bureau_balance_STATUS_1_count_norm_mean,client_bureau_balance_STATUS_0_count_norm_min,client_bureau_balance_STATUS_1_count_norm_max,client_bureau_balance_STATUS_1_count_norm_sum,client_bureau_balance_STATUS_1_count_mean,client_bureau_balance_STATUS_X_count_norm_mean,client_bureau_balance_STATUS_0_count_norm_mean,client_bureau_balance_STATUS_C_count_norm_mean,client_bureau_balance_STATUS_X_count_norm_max,client_bureau_balance_STATUS_C_count_norm_max,client_bureau_balance_STATUS_0_count_norm_max,client_bureau_balance_STATUS_0_count_min,client_bureau_balance_STATUS_1_count_max,client_bureau_balance_STATUS_1_count_sum,client_bureau_balance_STATUS_X_count_norm_sum,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_STATUS_0_count_norm_sum,client_bureau_balance_STATUS_C_count_norm_sum,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_0_count_mean,client_bureau_balance_MONTHS_BALANCE_sum_count,client_bureau_balance_STATUS_X_count_max,client_bureau_balance_STATUS_0_count_max,client_bureau_balance_STATUS_C_count_mean,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_STATUS_X_count_sum,client_bureau_balance_STATUS_0_count_sum,client_bureau_balance_STATUS_C_count_max,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_STATUS_C_count_sum,client_bureau_balance_MONTHS_BALANCE_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001.0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,-1,-0.5,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0.000000,0,0.007519,0.019231,0.052632,0.052632,0.142857,0.214590,0.336651,0.441240,0.500000,0.966667,1.000000,1,1,1,1.502129,2,2.356557,3.088683,4.285

### `previous application.csv`

In [13]:
prev = pd.read_csv('drive/MyDrive/CS 249 Project/Data/previous_application.csv')
prev = convert_types(prev, print_info=True)
prev.head()

Original Memory Usage: 0.49 gb.
New Memory Usage: 0.16 gb.


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,True,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,True,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,True,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,True,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,True,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Aggregate categorical and numerical data
prev_agg_num = agg_numeric(prev, 'SK_ID_CURR', 'prev')
prev_agg_cat = agg_categorical(prev, 'SK_ID_CURR', 'prev')
print(prev_agg_num.shape)
print(prev_agg_cat.shape)

(338857, 80)
(338857, 284)


## Merge computed features with training data

Note: so far only `bureau` and `bureau_balance` are being merged with the train
set. Still need to aggregate and merge the rest of the csv files.

In [15]:
print(f'Original num of features: {train.shape[1]}')
train = train.merge(bureau_agg_cat, on='SK_ID_CURR', how='left')
train = train.merge(bureau_agg_num, on='SK_ID_CURR', how='left')
train = train.merge(bureau_balance_by_client, on='SK_ID_CURR', how='left')
train = train.merge(prev_agg_num, on='SK_ID_CURR', how='left')
train = train.merge(prev_agg_cat, on='SK_ID_CURR', how='left')
print(f'Final num of features: {train.shape[1]}')

gc.enable()
del bureau_agg_cat, bureau_agg_num, bureau_bal_agg_cat, bureau_bal_agg_num
bureau_by_loan, bureau_balance_by_client, prev_agg_cat,
prev_agg_num, bureau, bureau_bal, prev
gc.collect()

Original num of features: 122
Final num of features: 673


0

In [16]:
# Remove features with too many missing values
train = del_feat_miss_val(train)

## Aggregate MORRRRREEEEE data

### `POS_CASH_balance.csv`
This file contains data for the monthly balance of a client's (identified by `SK_ID_CURR`) previous loans in Home Credit, as well as behavioral data (whatever that means).

In [17]:
pos_cash_bal = pd.read_csv('drive/MyDrive/CS 249 Project/Data/POS_CASH_balance.csv')
pos_cash_bal = convert_types(pos_cash_bal, print_info=True)
pos_cash_bal.head()

Original Memory Usage: 0.64 gb.
New Memory Usage: 0.29 gb.


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [18]:
def agg_prev_data(df, groupby_vars, df_names):
  
  # Aggregate numerical data
  df_agg = agg_numeric(df, groupby_vars[0], df_names[0])
  
  if any(df.dtypes == 'category'):
    # Aggregate categorical data
    df_agg_cat = agg_categorical(df, groupby_vars[0], df_names[0])

    # Merge numerical and categorical
    df_prev_loan = df_agg_cat.merge(df_agg, on=groupby_vars[0], how='outer')

    gc.enable()
    del df_agg, df_agg_cat
    gc.collect()

    # Merge with current loan ID
    df_prev_loan = df_prev_loan.merge(df[[groupby_vars[0], groupby_vars[1]]],
                                         on=groupby_vars[0], how='left')
  else:
    df_prev_loan = df_agg.merge(df[[groupby_vars[0], groupby_vars[1]]],
                                on=groupby_vars[0], how='left')
    gc.enable()
    del df_agg
    gc.collect()

  # Remove the prev loan ID and aggregate at the curr loan level
  df_prev_loan = df_prev_loan.drop(columns=groupby_vars[0])
  df_curr_loan = agg_numeric(df_prev_loan, groupby_vars[1], groupby_vars[1])

  gc.enable()
  del df, df_prev_loan
  gc.collect()

  return df_curr_loan

Use the function above to aggregate previous loan data for each current loan application based on `SK_ID_CURR`.

In [19]:
cash_curr_loan = agg_prev_data(
    pos_cash_bal,
    groupby_vars=['SK_ID_PREV', 'SK_ID_CURR'],
    df_names=['cash', 'client']
)
cash_curr_loan.head()

,SK_ID_CURR_cash_MONTHS_BALANCE_sum_sum,SK_ID_CURR_cash_MONTHS_BALANCE_min_sum,SK_ID_CURR_cash_MONTHS_BALANCE_mean_sum,SK_ID_CURR_cash_MONTHS_BALANCE_max_sum,SK_ID_CURR_cash_MONTHS_BALANCE_sum_min,SK_ID_CURR_cash_MONTHS_BALANCE_sum_mean,SK_ID_CURR_cash_MONTHS_BALANCE_sum_max,SK_ID_CURR_cash_MONTHS_BALANCE_min_min,SK_ID_CURR_cash_MONTHS_BALANCE_mean_min,SK_ID_CURR_cash_MONTHS_BALANCE_max_min,SK_ID_CURR_cash_MONTHS_BALANCE_min_mean,SK_ID_CURR_cash_MONTHS_BALANCE_mean_mean,SK_ID_CURR_cash_MONTHS_BALANCE_max_mean,SK_ID_CURR_cash_MONTHS_BALANCE_min_max,SK_ID_CURR_cash_MONTHS_BALANCE_mean_max,SK_ID_CURR_cash_MONTHS_BALANCE_max_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_norm_min,SK_ID_CURR_cash_SK_DPD_DEF_min_min,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_min,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_min,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_min,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Amortized debt_count_min,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_norm_mean,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_norm_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_mean,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_norm_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_sum,SK_ID_CURR_cash_SK_DPD_min_min,SK_ID_CURR_cash_SK_DPD_DEF_min_mean,SK_ID_CURR_cash_SK_DPD_DEF_min_max,SK_ID_CURR_cash_SK_DPD_DEF_min_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_mean,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_mean,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Canceled_count_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_mean,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_max,...,SK_ID_CURR_cash_CNT_INSTALMENT_min_max,SK_ID_CURR_cash_CNT_INSTALMENT_mean_max,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_max_max,SK_ID_CURR_cash_CNT_INSTALMENT_max_max,SK_ID_CURR_cash_CNT_INSTALMENT_count_mean,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_count_mean,SK_ID_CURR_cash_MONTHS_BALANCE_count_mean,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_count_max,SK_ID_CURR_cash_CNT_INSTALMENT_count_max,SK_ID_CURR_cash_MONTHS_BALANCE_count_max,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_sum_mean,SK_ID_CURR_cash_SK_DPD_DEF_max_max,SK_ID_CURR_cash_SK_DPD_DEF_sum_max,SK_ID_CURR_cash_SK_DPD_DEF_mean_sum,SK_ID_CURR_cash_SK_DPD_max_max,SK_ID_CURR_cash_SK_DPD_mean_sum,SK_ID_CURR_cash_SK_DPD_sum_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Active_count_norm_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Completed_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_min_count,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_norm_count,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_sum_max,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_mean_sum,SK_ID_CURR_cash_CNT_INSTALMENT_sum_min,SK_ID_CURR_cash_CNT_INSTALMENT_sum_mean,SK_ID_CURR_cash_CNT_INSTALMENT_sum_max,SK_ID_CURR_cash_SK_DPD_DEF_max_sum,SK_ID_CURR_cash_SK_DPD_DEF_sum_sum,SK_ID_CURR_cash_SK_DPD_max_sum,SK_ID_CURR_cash_SK_DPD_sum_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_max_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Active_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_min_sum,SK_ID_CURR_cash_CNT_INSTALMENT_mean_sum,SK_ID_CURR_cash_CNT_INSTALMENT_max_sum,SK_ID_CURR_cash_CNT_INSTALMENT_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_count_sum,SK_ID_CURR_cash_MONTHS_BALANCE_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_sum_sum,SK_ID_CURR_cash_CNT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-2887,-669,-653.0,-637,-378,-320.777778,-275,-96,-94.5,-93,-74.333333,-72.555556,-70.777778,-57,-55.0,-53,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,...,4.0,4.00,4.0,4.0,4.555556,4.555556,4.555556,5,5,5,6.888889,7,7,7.0,7,7.0,7,7.0,9.0,9,9,10.0,13.000000,16.0,18.222221,20.0

In [20]:
# Merge with train set
train = train.merge(cash_curr_loan, on='SK_ID_CURR', how='left')
train = del_feat_miss_val(train)
train

gc.enable()
del cash_curr_loan, pos_cash_bal
gc.collect()

0

### `credit_card_balance.csv`
Aggregate monthly credit card data and merge to train set

In [21]:
cc_bal = pd.read_csv('drive/MyDrive/CS 249 Project/Data/credit_card_balance.csv')
cc_bal = convert_types(cc_bal, print_info=True)
cc_bal.head()

Original Memory Usage: 0.71 gb.
New Memory Usage: 0.34 gb.


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,1800.0,1800.0,0.000000,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,2250.0,2250.0,60175.078125,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,2250.0,2250.0,26926.425781,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,11925.0,11925.0,224949.281250,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,27000.0,27000.0,443044.406250,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [22]:
cc_bal_curr_loan = agg_prev_data(
    cc_bal,
    groupby_vars=['SK_ID_PREV', 'SK_ID_CURR'],
    df_names=['credit', 'client']
)
cc_bal_curr_loan.head()

,SK_ID_CURR_credit_MONTHS_BALANCE_sum_sum,SK_ID_CURR_credit_MONTHS_BALANCE_min_sum,SK_ID_CURR_credit_MONTHS_BALANCE_mean_sum,SK_ID_CURR_credit_MONTHS_BALANCE_sum_min,SK_ID_CURR_credit_MONTHS_BALANCE_sum_mean,SK_ID_CURR_credit_MONTHS_BALANCE_sum_max,SK_ID_CURR_credit_MONTHS_BALANCE_max_sum,SK_ID_CURR_credit_MONTHS_BALANCE_min_min,SK_ID_CURR_credit_MONTHS_BALANCE_min_mean,SK_ID_CURR_credit_MONTHS_BALANCE_min_max,SK_ID_CURR_credit_MONTHS_BALANCE_mean_min,SK_ID_CURR_credit_MONTHS_BALANCE_mean_mean,SK_ID_CURR_credit_MONTHS_BALANCE_mean_max,SK_ID_CURR_credit_MONTHS_BALANCE_max_mean,SK_ID_CURR_credit_AMT_RECIVABLE_min_sum,SK_ID_CURR_credit_AMT_TOTAL_RECEIVABLE_min_sum,SK_ID_CURR_credit_AMT_RECIVABLE_min_min,SK_ID_CURR_credit_AMT_TOTAL_RECEIVABLE_min_min,SK_ID_CURR_credit_AMT_RECIVABLE_min_mean,SK_ID_CURR_credit_AMT_TOTAL_RECEIVABLE_min_mean,SK_ID_CURR_credit_AMT_RECIVABLE_min_max,SK_ID_CURR_credit_AMT_TOTAL_RECEIVABLE_min_max,SK_ID_CURR_credit_SK_DPD_min_mean,SK_ID_CURR_credit_SK_DPD_min_sum,SK_ID_CURR_credit_CNT_DRAWINGS_OTHER_CURRENT_min_sum,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_min_sum,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_norm_min,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_min,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Approved_count_norm_max,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Approved_count_norm_mean,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Approved_count_mean,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Approved_count_norm_sum,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Approved_count_sum,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_norm_mean,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_norm_max,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_mean,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_max,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_norm_sum,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Demand_count_sum,SK_ID_CURR_credit_NAME_CONTRACT_STATUS_Refused_count_norm_max,...,SK_ID_CURR_credit_CNT_DRAWINGS_OTHER_CURRENT_mean_max,SK_ID_CURR_credit_CNT_DRAWINGS_OTHER_CURRENT_max_min,SK_ID_CURR_credit_CNT_DRAWINGS_OTHER_CURRENT_max_mean,SK_ID_CURR_credit_CNT_DRAWINGS_OTHER_CURRENT_max_max,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_min,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_mean,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_mean_max,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_max_min,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_max_mean,SK_ID_CURR_credit_AMT_DRAWINGS_OTHER_CURRENT_max_max,SK_ID_CURR_credit_CNT_DRAWINGS_POS_CURRENT_mean_min,SK_ID_CURR_credit_CNT_DRAWINGS_POS_CURRENT_mean_mean,SK_ID_CURR_credit_CNT_DRAWINGS_POS_CURRENT_mean_max,SK_ID_CURR_credit_CNT_DRAWINGS_POS_CURRENT_max_min,SK_ID_CURR_credit_CNT_DRAWINGS_POS_CURRENT_max_mean,SK_ID_CURR_credit_CNT_DRAWINGS_POS_CURRENT_max_max,SK_ID_CURR_credit_AMT_DRAWINGS_POS_CURRENT_mean_min,SK_ID_CURR_credit_AMT_DRAWINGS_POS_CURRENT_mean_mean,SK_ID_CURR_credit_AMT_DRAWINGS_POS_CURRENT_mean_max,SK_ID_CURR_credit_AMT_DRAWINGS_POS_CURRENT_max_min,SK_ID_CURR_credit_AMT_DRAWINGS_POS_CURRENT_max_mean,SK_ID_CURR_credit_AMT_DRAWINGS_POS_CURRENT_max_max,SK_ID_CURR_credit_CNT_DRAWINGS_ATM_CURRENT_mean_min,SK_ID_CURR_credit_CNT_DRAWINGS_ATM_CURRENT_mean_mean,SK_ID_CURR_credit_CNT_DRAWINGS_ATM_CURRENT_mean_max,SK_ID_CURR_credit_CNT_DRAWINGS_ATM_CURRENT_max_min,SK_ID_CURR_credit_CNT_DRAWINGS_ATM_CURRENT_max_mean,SK_ID_CURR_credit_CNT_DRAWINGS_ATM_CURRENT_max_max,SK_ID_CURR_credit_AMT_DRAWINGS_ATM_CURRENT_mean_min,SK_ID_CURR_credit_AMT_DRAWINGS_ATM_CURRENT_mean_mean,SK_ID_CURR_credit_AMT_DRAWINGS_ATM_CURRENT_mean_max,SK_ID_CURR_credit_AMT_PAYMENT_CURRENT_mean_min,SK_ID_CURR_credit_AMT_PAYMENT_CURRENT_mean_mean,SK_ID_CURR_credit_AMT_PAYMENT_CURRENT_mean_max,SK_ID_CURR_credit_AMT_PAYMENT_CURRENT_max_min,SK_ID_CURR_credit_AMT_PAYMENT_CURRENT_max_mean,SK_ID_CURR_credit_AMT_PAYMENT_CURRENT_max_max,SK_ID_CURR_credit_AMT_DRAWINGS_ATM_CURRENT_max_min,SK_ID_CURR_credit_AMT_DRAWINGS_ATM_CURRENT_max_mean,SK_ID_CURR_credit_AMT_DRAWINGS_ATM_CURRE

In [23]:
# Merge with train set
train = train.merge(cc_bal_curr_loan, on='SK_ID_CURR', how='left')

gc.enable()
del cc_bal, cc_bal_curr_loan
gc.collect()

0

Remove features with more than 70% of missing data to optimize memory usage.

In [24]:
train = del_feat_miss_val(train)
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,SK_ID_CURR_cash_CNT_INSTALMENT_min_max,SK_ID_CURR_cash_CNT_INSTALMENT_mean_max,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_max_max,SK_ID_CURR_cash_CNT_INSTALMENT_max_max,SK_ID_CURR_cash_CNT_INSTALMENT_count_mean,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_count_mean,SK_ID_CURR_cash_MONTHS_BALANCE_count_mean,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_count_max,SK_ID_CURR_cash_CNT_INSTALMENT_count_max,SK_ID_CURR_cash_MONTHS_BALANCE_count_max,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_sum_mean,SK_ID_CURR_cash_SK_DPD_DEF_max_max,SK_ID_CURR_cash_SK_DPD_DEF_sum_max,SK_ID_CURR_cash_SK_DPD_DEF_mean_sum,SK_ID_CURR_cash_SK_DPD_max_max,SK_ID_CURR_cash_SK_DPD_mean_sum,SK_ID_CURR_cash_SK_DPD_sum_max,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Active_count_norm_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Completed_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_min_count,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_XNA_count_norm_count,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_sum_max,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_mean_sum,SK_ID_CURR_cash_CNT_INSTALMENT_sum_min,SK_ID_CURR_cash_CNT_INSTALMENT_sum_mean,SK_ID_CURR_cash_CNT_INSTALMENT_sum_max,SK_ID_CURR_cash_SK_DPD_DEF_max_sum,SK_ID_CURR_cash_SK_DPD_DEF_sum_sum,SK_ID_CURR_cash_SK_DPD_max_sum,SK_ID_CURR_cash_SK_DPD_sum_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_max_sum,SK_ID_CURR_cash_NAME_CONTRACT_STATUS_Active_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_min_sum,SK_ID_CURR_cash_CNT_INSTALMENT_mean_sum,SK_ID_CURR_cash_CNT_INSTALMENT_max_sum,SK_ID_CURR_cash_CNT_INSTALMENT_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_count_sum,SK_ID_CURR_cash_MONTHS_BALANCE_count_sum,SK_ID_CURR_cash_CNT_INSTALMENT_FUTURE_sum_sum,SK_ID_CURR_cash_CNT_INSTALMENT_sum_sum
0,100002,True,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,True,True,0,True,True,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,...,24.0,24.000000,24.0,24.0,19.000000,19.000000,19.000000,19.0,19.0,19.0,285.000000,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,19.0,19.0,285.0,285.000000,456.0,456.000000,456.0,0.0,0.0,0.0,0.0,456.0,361.0,456.0,456.000,456.0,361.0,361.0,361.0,5415.0,8664.0
1,100003,False,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,True,True,0,True,True,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,...,12.0,12.000000,12.0,12.0,9.714286,9.714286,9.714286,12.0,12.0,12.0,57.428570,0.0,0.0,0.0,0.0,0.0,0.0,26.0,16.0,28.0,28.0,78.0,162.000000,48.0,101.428574,144.0,0.0,0.0,0.0,0.0,288.0,256.0,248.0,283.000,288.0,272.0,272.0,272.0,1608.0,2840.0
2,100004,False,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,True,True,1,True,True,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,...,3.0,3.750000,4.0,4.0,4.000000,4.000000,4.000000,4.0,4.0,4.0,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,4.0,4.0,9.0,9.000000,15.0,15.000000,15.0,0.0,0.0,0.0,0.0,16.0,12.0,12.0,15.000,16.0,16.0,16.0,16.0,36.0,60.0
3,100006,False,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary spec

### `installments_payments.csv`

In [25]:
installments = pd.read_csv('drive/MyDrive/CS 249 Project/Data/installments_payments.csv')
installments = convert_types(installments, print_info=True)
installments.head()

Original Memory Usage: 0.87 gb.
New Memory Usage: 0.44 gb.


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [27]:
installments_curr_loan = agg_prev_data(
    installments,
    groupby_vars=['SK_ID_PREV', 'SK_ID_CURR'],
    df_names=['installments', 'client']
)
installments_curr_loan.head()

,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_sum_sum,SK_ID_CURR_installments_DAYS_INSTALMENT_sum_sum,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_min_sum,SK_ID_CURR_installments_DAYS_INSTALMENT_min_sum,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_mean_sum,SK_ID_CURR_installments_DAYS_INSTALMENT_mean_sum,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_max_sum,SK_ID_CURR_installments_DAYS_INSTALMENT_max_sum,SK_ID_CURR_installments_DAYS_INSTALMENT_sum_min,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_sum_min,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_sum_mean,SK_ID_CURR_installments_DAYS_INSTALMENT_sum_mean,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_sum_max,SK_ID_CURR_installments_DAYS_INSTALMENT_sum_max,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_min_min,SK_ID_CURR_installments_DAYS_INSTALMENT_min_min,SK_ID_CURR_installments_DAYS_INSTALMENT_mean_min,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_mean_min,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_max_min,SK_ID_CURR_installments_DAYS_INSTALMENT_max_min,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_min_mean,SK_ID_CURR_installments_DAYS_INSTALMENT_min_mean,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_mean_mean,SK_ID_CURR_installments_DAYS_INSTALMENT_mean_mean,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_max_mean,SK_ID_CURR_installments_DAYS_INSTALMENT_max_mean,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_min_max,SK_ID_CURR_installments_DAYS_INSTALMENT_min_max,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_mean_max,SK_ID_CURR_installments_DAYS_INSTALMENT_mean_max,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_max_max,SK_ID_CURR_installments_DAYS_INSTALMENT_max_max,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_min_min,SK_ID_CURR_installments_NUM_INSTALMENT_NUMBER_min_min,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_min_mean,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_min_max,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_mean_min,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_max_min,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_mean_mean,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_mean_max,...,SK_ID_CURR_installments_NUM_INSTALMENT_NUMBER_sum_max,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_max_sum,SK_ID_CURR_installments_NUM_INSTALMENT_NUMBER_mean_sum,SK_ID_CURR_installments_DAYS_ENTRY_PAYMENT_count_sum,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_count_sum,SK_ID_CURR_installments_NUM_INSTALMENT_NUMBER_max_sum,SK_ID_CURR_installments_NUM_INSTALMENT_VERSION_sum_sum,SK_ID_CURR_installments_NUM_INSTALMENT_NUMBER_sum_sum,SK_ID_CURR_installments_AMT_PAYMENT_min_min,SK_ID_CURR_installments_AMT_INSTALMENT_min_min,SK_ID_CURR_installments_AMT_PAYMENT_min_mean,SK_ID_CURR_installments_AMT_INSTALMENT_min_mean,SK_ID_CURR_installments_AMT_PAYMENT_min_max,SK_ID_CURR_installments_AMT_INSTALMENT_min_max,SK_ID_CURR_installments_AMT_PAYMENT_mean_min,SK_ID_CURR_installments_AMT_INSTALMENT_mean_min,SK_ID_CURR_installments_AMT_INSTALMENT_max_min,SK_ID_CURR_installments_AMT_PAYMENT_max_min,SK_ID_CURR_installments_AMT_PAYMENT_mean_mean,SK_ID_CURR_installments_AMT_INSTALMENT_mean_mean,SK_ID_CURR_installments_AMT_PAYMENT_mean_max,SK_ID_CURR_installments_AMT_INSTALMENT_mean_max,SK_ID_CURR_installments_AMT_INSTALMENT_max_mean,SK_ID_CURR_installments_AMT_PAYMENT_max_mean,SK_ID_CURR_installments_AMT_PAYMENT_sum_min,SK_ID_CURR_installments_AMT_INSTALMENT_sum_min,SK_ID_CURR_installments_AMT_INSTALMENT_max_max,SK_ID_CURR_installments_AMT_PAYMENT_max_max,SK_ID_CURR_installments_AMT_PAYMENT_sum_mean,SK_ID_CURR_installments_AMT_INSTALMENT_sum_mean,SK_ID_CURR_installments_AMT_PAYMENT_min_sum,SK_ID_CURR_installments_AMT_INSTALMENT_min_sum,SK_ID_CURR_installments_AMT_PAYMENT_sum_max,SK_ID_CURR_installments_AMT_INSTALMENT_sum_max,SK_ID_CURR_installments_AMT_PAYMENT_mean_sum,SK_ID_CURR_installments_AMT_INSTALMENT_mean_sum,SK_ID_CURR_installments_AMT_INSTALMENT_max_sum,SK_ID_CURR_installments_AMT_PAYMENT_max_sum,SK_ID_CURR_installments_AMT_PAYMENT_sum_sum,SK_ID_CURR_installments_AMT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [28]:
train = train.merge(installments_curr_loan, on='SK_ID_CURR', how='left')
gc.enable
del installments_curr_loan, installments
gc.collect()

0

In [29]:
train.shape

(307511, 809)